<a href="https://colab.research.google.com/github/jaihonikhil/Group-15/blob/Jaya/Processing_Code_For_Task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PREPROCESSING CODE FOR ONE XML FILE

In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/Autogenerated Annotations.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/My Drive')

In [11]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/Manual Annotations.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/My Drive')

In [165]:
from os import walk
mypathTrainI="/content/drive/My Drive/Autogenerated Annotations/Train/input"
mypathTrainO="/content/drive/My Drive/Autogenerated Annotations/Train/output"
mypathDevI="/content/drive/My Drive/Autogenerated Annotations/Dev/input"
mypathDevO="/content/drive/My Drive/Autogenerated Annotations/Dev/output"
# mypathDevO="/content/drive/My Drive/Manual Annotations/Dev/output"

In [166]:
import os
files_in_directory = os.listdir(mypathDevO)#EDIT THIS FOR ANOTHER DIRECTORY
examples = [file for file in files_in_directory if file.endswith(".xml")]
for e in range(len(examples)):
  examples[e]=examples[e][:-4]


In [168]:
from bs4 import BeautifulSoup as bs
import numpy as np
import json
import csv 
filename=[]
stateme=[]
ind=[]
captio=[]
for example1 in examples:
  content = []
  with open(mypathDevO+"/"+example1+".xml", "r") as file:#EDIT THIS FOR ANOTHER DIRECTORY
      content = file.readlines()
      content = "".join(content)
      soup = bs(content, "lxml")
  matrices=[]
  tables = soup.find_all('table')
  num_tables = len(tables)
  p=0
  
  for t in tables:
    p=p+1  
    row=t.find_all('row')
    capt=t.find('caption')
    statements=t.find_all('statement')
    state=[]
    index=[]
    for s in statements:
      a_list = s['text'].split()
      s['text'] = " ".join(a_list)
      if (s['type']=='entailed'):
        state.append(s['text'])
        s['type']='1'
        index.append(s['type'])
      elif (s['type']=='refuted'):
        state.append(s['text'])
        s['type']='0'
        index.append(s['type'])
    
    stateme.append(state)
    ind.append(index)
    if (capt!=None):
      a_list = capt['text'].split()
      capt['text'] = " ".join(a_list)
      captio.append(capt['text'])
    else:
      captio.append(" ")
    cells=[]
    for s in row:
      table_s = s.find_all('cell')
      cells.append(len(table_s))
    m=max(cells)# For intrapolating hence finding maximum number of columns
    matrix=[]
    lex=[]
    for r in row:
      table_rows = r.find_all('cell')
      j=0
      x=0
      s=""
      rowstring=""
      for j in range(m):
        l=table_rows[x]
        if(j==int(l['col'])):
          rowstring=rowstring+l['text']+'#'
          s=l['text']
          if(x<len(table_rows)-1):
            x=x+1
        else :
          rowstring=rowstring+s+'#'
        j=j+1
      rowstring=rowstring[:-1]
      a_list = rowstring.split()
      new_string = " ".join(a_list)
      lex.append(new_string)
      matrix.append(lex)
      lex=[]
      s=""
      rowstring=""
    filename.append(example1+"."+str(p)+".html.csv")
    dir="/content/drive/My Drive/DevOutput/"#EDIT THIS FOR ANOTHER DIRECTORY
    file = open(dir+example1+"."+str(p)+'.html.csv', 'w', newline ='') 
    with file:     
      write = csv.writer(file) 
      write.writerows(matrix) 
    matrices.append(matrix)


In [169]:
#Creates File For Json Writing 
feeds=[]
with open("/content/drive/My Drive/Group15Data/Devoutput.json", "w") as write_file:#EDIT THIS FOR ANOTHER DIRECTORY
  for i in range(len(filename)):
    a_list = captio[i].split()
    new_string = " ".join(a_list)
    entry = {filename[i]:[stateme[i], ind[i],new_string]}
    feeds.append(entry)
    i=i+1
  json.dump(feeds, write_file,indent=4)
#Json writing ends